In [1]:
# Inspecting GBIF CSV File
# Step 1: Import the pandas library
import pandas as pd


# Step 2: Load the CSV file into a DataFrame
GBIF_df = pd.read_csv(r'C:\Users\Walter\Documents\GitHub\FeatheredMaps\0010953-250127130748423.csv', delimiter='\t')

# Step 3: Check the number of rows in the DataFrame
num_rows = len(GBIF_df)
print(f"Number of rows: {num_rows}")

# Step 4: Count the number of unique entries in the 'species' column
unique_species = GBIF_df['species'].nunique()

print(f"Number of unique entries in 'species': {unique_species}")

Number of rows: 864
Number of unique entries in 'species': 628


C:\Users\Walter\AppData\Local\Temp\ipykernel_20104\1816870567.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Dropping some columns and dropping rows where observation is incomplete eg. only genus is known and sort the file 
# List of columns to drop
columns_to_drop = ['kingdom', 'kingdomKey', 'phylum', 'phylumKey', 'class', 'classKey','taxonKey', 'acceptedTaxonKey']
GBIF_df = GBIF_df.drop(columns=columns_to_drop)


# Drop everything that does not mention at least species or subspecies
# Those are reports were the species was not known
smaller_GBIF_df = GBIF_df[GBIF_df['taxonRank'].str.upper().isin(['SPECIES', 'SUBSPECIES'])]
rows_not_species = GBIF_df[~GBIF_df['taxonRank'].str.upper().isin(['SPECIES', 'SUBSPECIES'])]
print(rows_not_species[["scientificName","taxonRank","numberOfOccurrences"]])

print(f"Number of rows in removed slice should be: {len(GBIF_df)-len(smaller_GBIF_df)} and is {len(rows_not_species)}")

# Sort the DataFrame by multiple columns
sorted_GBIF_df = smaller_GBIF_df.sort_values(
    by=['orderKey', 'familyKey', 'genusKey', 'speciesKey', 'numberOfOccurrences'],
    ascending=[True, True, True, True, False]
)

# Save the sorted DataFrame to a new CSV file (optional)
sorted_GBIF_df.to_csv('1_GBIF_sorted_file.csv', index=False)
print("Sorted DataFrame saved to '1_GBIF_sorted_file.csv'.")

                                    scientificName taxonRank  \
60                            Astur Lacepède, 1799     GENUS   
66                      Pteroglossus Illiger, 1811     GENUS   
227                      Tyranniscus Cabanis, 1851     GENUS   
320                     Columba livia f. domestica      FORM   
436                                           Aves     CLASS   
516  Columba livia var. domestica J.F.Gmelin, 1789   VARIETY   
550                      Ramphastos Linnaeus, 1758     GENUS   
585              Percnostola Cabanis & Heine, 1859     GENUS   
843                         Scytalopus Gould, 1837     GENUS   

     numberOfOccurrences  
60                     5  
66                     1  
227                    1  
320                    3  
436                   27  
516                   15  
550                    1  
585                   11  
843                    1  
Number of rows in removed slice should be: 9 and is 9
Sorted DataFrame saved to '1_GBIF_s

In [3]:
# Extracting the species and subspecies names for lookup in the next stage
# Split the string at the second space
syn_split_columns = sorted_GBIF_df["scientificName"].str.split(" ", n=2, expand=True)
split_columns = sorted_GBIF_df["acceptedScientificName"].str.split(" ", n=2, expand=True)

# Create the new columns
sorted_GBIF_df["synScientificName"] = syn_split_columns[0] + " " + syn_split_columns[1]
sorted_GBIF_df["accScientificName"] = split_columns[0] + " " + split_columns[1]
sorted_GBIF_df["synSplitExtras"] = syn_split_columns[2]
sorted_GBIF_df["accSplitExtras"] = split_columns [2]


# Function to modify splitExtras if taxonRank is SUBSPECIES
def adjust_for_syn(row):
    ssE = row["synSplitExtras"]
    if ssE and ssE[0].islower() and ssE[1].islower():
        # Split the splitExtras column at the first space
        split_extras = ssE.split(" ", 1)
        # Append the first part to the splitScientificName
        new_split_scientific_name = row["synScientificName"] + " " + split_extras[0]
        # Rebuild the splitExtras with the remaining part (if any)
        new_split_extras = split_extras[1] if len(split_extras) > 1 else ""
        return pd.Series([new_split_scientific_name, new_split_extras])
    else:
        # Return the original values if taxonRank is not SUBSPECIES
        return pd.Series([row["synScientificName"], row["synSplitExtras"]])
    
# Function to modify splitExtras if taxonRank is SUBSPECIES
def adjust_for_acc(row):
    aSE = row["accSplitExtras"]
    if aSE and aSE[0].islower() and aSE[1].islower():
        # Split the splitExtras column at the first space
        split_extras = aSE.split(" ", 1)
        # Append the first part to the splitScientificName
        new_split_scientific_name = row["accScientificName"] + " " + split_extras[0]
        # Rebuild the splitExtras with the remaining part (if any)
        new_split_extras = split_extras[1] if len(split_extras) > 1 else ""
        return pd.Series([new_split_scientific_name, new_split_extras])
    else:
        # Return the original values if taxonRank is not SUBSPECIES
        return pd.Series([row["accScientificName"], row["accSplitExtras"]])

# Apply the function to each row
sorted_GBIF_df[["synScientificName", "synSplitExtras"]] = sorted_GBIF_df.apply(adjust_for_syn, axis=1)
sorted_GBIF_df[["accScientificName", "accSplitExtras"]] = sorted_GBIF_df.apply(adjust_for_acc, axis=1)


# Process the DataFrame
for index, row in sorted_GBIF_df.iterrows():
    if row["accScientificName"] == row["synScientificName"]:
        sorted_GBIF_df.at[index, "synScientificName"] = ""
        sorted_GBIF_df.at[index, "synSplitExtras"] = ""

indexes_to_drop = []
for index1, row1 in sorted_GBIF_df.iterrows():
    if row1["taxonomicStatus"] == "SYNONYM":
        for index2, row2 in sorted_GBIF_df.iterrows():
            if (row1["accScientificName"] == row2["accScientificName"]) and (row2["taxonomicStatus"] == "ACCEPTED"):
                indexes_to_drop.append(index1)
                sorted_GBIF_df.at[index2, "numberOfOccurrences"] += row1['numberOfOccurrences']
                if row2["synScientificName"] == "":
                    sorted_GBIF_df.at[index2, "synScientificName"] += row1['synScientificName']
                    sorted_GBIF_df.at[index2, "synSplitExtras"] += row1['synSplitExtras']
                else:
                    sorted_GBIF_df.at[index2, "synScientificName"] += f"; {row1['synScientificName']}"
                    sorted_GBIF_df.at[index2, "synSplitExtras"] += f"; {row1['synSplitExtras']}"
                if (row2["iucnRedListCategory"] == "") or (row2["iucnRedListCategory"] == "NE"):
                    sorted_GBIF_df.at[index2, "iucnRedListCategory"] = row1['iucnRedListCategory']
sorted_GBIF_df = sorted_GBIF_df.drop(indexes_to_drop) 

indexes_to_drop = []
for index1, row1 in sorted_GBIF_df.iterrows():
    if row1["taxonomicStatus"] == "SYNONYM":
        for index2, row2 in sorted_GBIF_df.iterrows():
            if (row1["accScientificName"] == row2["accScientificName"]) and (row2["taxonomicStatus"] == "SYNONYM") and (index1 != index2):
                if row1["numberOfOccurrences"] >= row2['numberOfOccurrences']:
                    sorted_GBIF_df.at[index1, "numberOfOccurrences"] += row2['numberOfOccurrences']
                    indexes_to_drop.append(index2)
                    if row1["synScientificName"] == "":
                        sorted_GBIF_df.at[index1, "synScientificName"] += row2['synScientificName']
                        sorted_GBIF_df.at[index1, "synSplitExtras"] += row2['synSplitExtras']
                    else:
                        sorted_GBIF_df.at[index1, "synScientificName"] += f"; {row2['synScientificName']}"
                        sorted_GBIF_df.at[index1, "synSplitExtras"] += f"; {row2['synSplitExtras']}"
                    if (row1["iucnRedListCategory"] == "") or (row1["iucnRedListCategory"] == "NE"):
                        sorted_GBIF_df.at[index1, "iucnRedListCategory"] = row2['iucnRedListCategory']
                else:
                    sorted_GBIF_df.at[index2, "numberOfOccurrences"] += row1['numberOfOccurrences']
                    indexes_to_drop.append(index1)
                    if row2["synScientificName"] == "":
                        sorted_GBIF_df.at[index2, "synScientificName"] += row1['synScientificName']
                        sorted_GBIF_df.at[index2, "synSplitExtras"] += row1['synSplitExtras']
                    else:
                            sorted_GBIF_df.at[index2, "synScientificName"] += f"; {row1['synScientificName']}"
                            sorted_GBIF_df.at[index2, "synSplitExtras"] += f"; {row1['synSplitExtras']}"
                    if (row2["iucnRedListCategory"] == "") or (row2["iucnRedListCategory"] == "NE"):
                        sorted_GBIF_df.at[index2, "iucnRedListCategory"] = row1['iucnRedListCategory']
sorted_GBIF_df = sorted_GBIF_df.drop(indexes_to_drop) 

def remove_dublicate_syn(row):
    try:
        parts = row["synScientificName"].split("; ")
        partsExtras = row["synSplitExtras"].split("; ")
        if len(parts) == len(partsExtras):
            list_df = pd.DataFrame({
                'Column1': parts,
                'Column2': partsExtras
                })
            # Step 2: Identify duplicates in Column1 and keep the first occurrence
            df_cleaned = list_df.drop_duplicates(subset='Column1', keep='first')
            parts_new = df_cleaned['Column1'].tolist()
            partsExtras_new = df_cleaned['Column2'].tolist()
            parts_new = "; ".join(parts_new)
            partsExtras_new = "; ".join(partsExtras_new)
            return pd.Series([parts_new, partsExtras_new])

    except:
        return pd.Series([row["synScientificName"], row["synSplitExtras"]])
        


sorted_GBIF_df[["synScientificName", "synSplitExtras"]] = sorted_GBIF_df.apply(remove_dublicate_syn, axis=1)



# Drop the original column if desired
#sorted_GBIF_df = sorted_GBIF_df.drop(columns=["scientificName"])

# Save the modified DataFrame to a new CSV file (optional)
sorted_GBIF_df.to_csv('2_newScientificNames.csv', index=False)
print("Modified DataFrame saved to '2_newScientificNames.csv'.")

Modified DataFrame saved to '2_newScientificNames.csv'.


In [161]:
print(len(sorted_GBIF_df))

821


In [68]:
mask = (sorted_GBIF_df["synScientificName"] == "") & (sorted_GBIF_df["taxonomicStatus"] == "SYNONYM")

# Check if any such row exists
if mask.any():
    print("There are rows where synScientificName is not empty but taxonomicStatus is not SYNONYM.")
else:
    print("No such rows found.")


No such rows found.


In [162]:
lookup_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\Further_Combined_Clements_IOC.csv")

# List to store concatenated rows
merged_rows = []
unmerged_rows = []
i = 0
# Iterate through df1
for index1, row1 in sorted_GBIF_df.iterrows():
    i += 1
    print(i)
    MatchFound = False
    # Iterate through df2
    synonym_list = []
    name_list = []
    synonyms = row1["synScientificName"]
    synonym_list = synonyms.split("; ")
    name_list = [row1["accScientificName"]]
    name_list.extend(synonym_list)
    for index2, row2 in lookup_df.iterrows():
        scientificName_IOC = row2["scientificName_IOC"]
        scientificName_Clements = row2["scientificName_Clements"]
        if (scientificName_IOC in name_list) or (scientificName_Clements in name_list):  # Check conditions
            MatchFound = True
            merged_row = pd.concat([row1, row2], axis=0)  # Concatenate rows horizontally
            merged_rows.append(merged_row)
            break  # Stop checking after first match
    if MatchFound != True:
        unmerged_rows.append(row1)
  
        

# Create a new DataFrame from merged rows
merged_df = pd.DataFrame(merged_rows)

#print(merged_df)
merged_df.to_csv("2a_merged_df.csv")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [164]:
print(len(unmerged_rows))
print(unmerged_rows)

25
[scientificName            Pachyramphus albogriseus guayaquilensis Zimmer...
acceptedScientificName    Pachyramphus albogriseus guayaquilensis Zimmer...
numberOfOccurrences                                                       5
taxonRank                                                        SUBSPECIES
taxonomicStatus                                                    ACCEPTED
order                                                         Passeriformes
orderKey                                                              729.0
family                                                           Cotingidae
familyKey                                                            5236.0
genus                                                          Pachyramphus
genusKey                                                          2486256.0
species                                            Pachyramphus albogriseus
speciesKey                                                        5230352.0
iucnRedL

In [150]:
string = "Hello1"
parts = string.split("; ")
print(parts)

hello_list = ['Hello2']
hello_list.extend(parts)
print(hello_list)

['Hello1']
['Hello2', 'Hello1']


In [52]:
lookup_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\Further_Combined_Clements_IOC.csv")
merged_df_IOC = pd.merge(sorted_GBIF_df,lookup_df,left_on="splitScientificName",right_on="scientificName_IOC",how="left")
merged_df_Clements = pd.merge(sorted_GBIF_df,lookup_df,left_on="splitScientificName",right_on="scientificName_Clements",how="left")

# Append the new rows to df1
merged_df = pd.concat([merged_df_IOC, merged_df_Clements], ignore_index=True)
# Remove duplicate rows
merged_df = merged_df.drop_duplicates()

#unmerged_df = sorted_GBIF_df[~sorted_GBIF_df.apply(tuple, axis=1).isin(merged_df.apply(tuple, axis=1))]



# Sort the DataFrame by multiple columns
sorted_GBIF_df = smaller_GBIF_df.sort_values(
    by=['orderKey', 'familyKey', 'genusKey', 'speciesKey', 'numberOfOccurrences'],
    ascending=[True, True, True, True, False]
)

merged_df.to_csv("2a_merged_df.csv")
#unmerged_df.to_csv("2a_unmerged_df.csv")


In [ ]:
# Looking up scientific names on Wikipedia and searching for Common Name
import requests
from bs4 import BeautifulSoup

try:
    wiki_lookup_dict_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3a_wiki_lookup_table.csv")
except:
    wiki_lookup_dict_df = pd.DataFrame(columns=["gbif_scientificName", "wiki_commonName"])
    wiki_lookup_dict = {}
# Convert the DataFrame into a dictionary


# Construct the dictionary

for index, row in df.iterrows():
    key = row.iloc[0]  # First column as the key
    # Check for the second column or the first non-empty value in other columns
    value = row.iloc[1] if pd.notna(row.iloc[1]) else next((val for val in row.iloc[2:] if pd.notna(val)), None)



total = len(sorted_GBIF_df)


# Function to get the common name from Wikipedia
def get_common_name(scientific_name):

    scientific_name_full = scientific_name
    CommonNameFound = False
    NewNameFound = True
    if not hasattr(get_common_name, "counter"):
        get_common_name.counter = 0
    # Increment the counter
    get_common_name.counter += 1
    
    while True:
        if scientific_name in wiki_lookup_dict: #species or subspecies already in dict
            CommonName = wiki_lookup_dict[scientific_name]
            print(f"    {get_common_name.counter}/{total}    Found {CommonName} for {scientific_name} in wiki_lookup")
            CommonNameFound = True
            NewNameFound = False
            break

        else: #species or subspecies not in dict yet
            print(f"    Inquiring about {scientific_name}")
            url = f"https://en.wikipedia.org/wiki/{scientific_name.replace(' ', '_')}"
            response = requests.get(url)

            if response.status_code == 200: #postive response from server
                soup = BeautifulSoup(response.content, "html.parser")
                # Extract the page title
                title_tag = soup.find("span", class_="mw-page-title-main")
                if title_tag:
                    CommonName = title_tag.text.strip()
                    if CommonName.lower() != scientific_name.lower():
                        CommonNameFound = True
                        print(f"    {get_common_name.counter}/{total}    Found \033[1m{CommonName}\033[0m for \033[1m{scientific_name}\033[0m")
                        break
                    else:
                        print(f"Common name not found in the title for {scientific_name}.")
                        CommonNameFound = False
                        parts = scientific_name.split(' ')
                        if len(parts) >= 3:
                            scientific_name = ' '.join(parts[:-1])
                        else:
                            print(f"Page not found for {scientific_name}.")
                            break
                else:
                    print(f"Something wrong with title_tag for {scientific_name}.")
                    CommonNameFound = False
                    break
            else: # negative or no response from server
                parts = scientific_name.split(' ')
                if len(parts) >= 3: #if not species try by searching for species
                    print(f"    Page not found for {scientific_name}, trying with species name")
                    scientific_name = ' '.join(parts[:-1])
                    CommonNameFound = False
                else: # negative response from server but is not a subspecies
                    print(f"Page not found for {scientific_name}.")
                    CommonNameFound = False
                    break
    if CommonNameFound:
        # Step 1: Title-case the string
        CommonName = CommonName.title()  # Result: "Red-Tailed Hawk"

        # Step 2: Lowercase the first word after the first dash
        if "-" in CommonName:
            string_split_by_dash = CommonName.split("-", 1)  # Split into two parts at the first dash
            CommonName = string_split_by_dash[0] + "-" + string_split_by_dash[1].lower() 
            string_split_by_space = CommonName.split(" ",1) 
            CommonName = string_split_by_space[0] + " " + string_split_by_space[1].title()

        if NewNameFound:
            wiki_lookup_dict[scientific_name_full] = CommonName
        return CommonName
    else:
        return None
    
# Example list of scientific names
# scientific_names = ["Struthio camelus", "Passer domesticus","Pteroglossus erythropygius","Dryobates callonotus","Veniliornis callonotus","Pteroglossus torquatus sanguineus","Chamaepetes goudotii fagani C.Chubb, 1917"]

sorted_GBIF_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\2_newScientificNames.csv")

# Apply the function to the 'splitScientificNames' column and create a new 'commonNames' column
sorted_GBIF_df["commonName"] = sorted_GBIF_df["splitScientificName"].apply(get_common_name)

# Convert the dictionary into a pandas DataFrame
wiki_lookup_dict_df = pd.DataFrame(list(wiki_lookup_dict.items()), columns=["gbif_scientificName", "wiki_commonName"])

# Save the DataFrame to a CSV file
wiki_lookup_dict_df.to_csv("3a_wiki_lookup_table.csv", index=False)

sorted_GBIF_df.to_csv("3_GBIF_named_file.csv", index=False)
print("\033[1mFinished inquiring all common names\033[0m")




    Inquiring about Chamaepetes goudotii
    1/855    Found Sickle-winged guan for Chamaepetes goudotii
    Inquiring about Chamaepetes goudotii fagani
    Page not found for Chamaepetes goudotii fagani, trying with species name
    2/855    Found Sickle-winged Guan for Chamaepetes goudotii in wiki_lookup
    Inquiring about Penelope montagnii
    3/855    Found Andean guan for Penelope montagnii
    Inquiring about Penelope ortoni
    4/855    Found Baudo guan for Penelope ortoni
    Inquiring about Penelope purpurascens
    5/855    Found Crested guan for Penelope purpurascens
    Inquiring about Aburria aburri


KeyboardInterrupt: 

In [2]:
named_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3_GBIF_named_file.csv")
wiki_lookup_dict_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3a_wiki_lookup_table.csv")

missing_common_name_rows = named_df[named_df["commonName"].isnull() | (named_df["commonName"] == "")]

# Print results

print(f"Number of rows in total: {len(named_df)}")
print(f"Number of rows without common names: {len(missing_common_name_rows)}")
print("Missing entries belong to the following species and subspecies:")
print("SplitScientificNames and their number of occurrences:")
print(missing_common_name_rows[["splitScientificName","taxonRank","numberOfOccurrences"]])


# Loop through each row and ask for input
for index, row in missing_common_name_rows.iterrows():
    user_input = input(f"Common name for: {row['splitScientificName']}")  # User provides input
    missing_common_name_rows.at[index, "manual_commonName"] = user_input  # Save input in column2


# Rename columns in the new data to match the existing DataFrame
missing_common_name_rows = missing_common_name_rows.rename(columns={
    "splitScientificName": "gbif_scientificName",
    "manual_commonName": "manual_commonName"
})

# Add a missing column in the new data for consistency (if needed)
if "gbif_commonName" not in missing_common_name_rows.columns:
    missing_common_name_rows["gbif_commonName"] = None  # Placeholder for missing data


# Append the DataFrame
wiki_lookup_dict_df = pd.concat([wiki_lookup_dict_df, missing_common_name_rows], ignore_index=True)

# Save the DataFrame to a CSV file
wiki_lookup_dict_df.to_csv("3a_wiki_lookup_table.csv", index=False)



Number of rows in total: 855
Number of rows without common names: 7
Missing entries belong to the following species and subspecies:
SplitScientificNames and their number of occurrences:
             splitScientificName taxonRank  numberOfOccurrences
22              Dryobates dignus   SPECIES                    6
127  Xiphorhynchus aequatorialis   SPECIES                    2
337       Phylloscartes flaveola   SPECIES                    2
411            Tangara aurulenta   SPECIES                    4
415             Tangara lunigera   SPECIES                    1
424          Buthraupis edwardsi   SPECIES                    2
467       Sporophila ophthalmica   SPECIES                    2


C:\Users\Walter\AppData\Local\Temp\ipykernel_9268\2835554811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_common_name_rows.at[index, "manual_commonName"] = user_input  # Save input in column2


In [154]:
# Combining all rows by common name and adding numberOfOccurrences, tallying up scientific names

named_GBIF_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3_GBIF_named_file.csv")

# Add the "Subspecies" column
named_GBIF_df["Subspecies"] = ""

# Process the DataFrame
for index, row in named_GBIF_df.iterrows():
    if row["taxonRank"] == "SUBSPECIES":
        # Find the row where splitScientificName matches the species entry
        species_match = named_GBIF_df[named_GBIF_df["splitScientificName"] == row["species"]]
        if not species_match.empty:
            # Get the index of the matched row
            matched_index = species_match.index[0]
            # Append the splitScientificName to the Subspecies column
            Species_nOO = named_GBIF_df.at[matched_index, "numberOfOccurrences"]
            sub_species_info = f"{row['splitScientificName']}, {row['numberOfOccurrences']/Species_nOO}"
            if named_GBIF_df.at[matched_index, "Subspecies"] is None:
                named_GBIF_df.at[matched_index, "Subspecies"] = f"{sub_species_info}"
            else:
                named_GBIF_df.at[matched_index, "Subspecies"] += f"; {sub_species_info}"
            named_GBIF_df.at[matched_index, "numberOfOccurrences"] += row['numberOfOccurrences']



# Drop rows where "taxonRank" contains "SUBSPECIES"
named_GBIF_df = named_GBIF_df[named_GBIF_df["taxonRank"] != "SUBSPECIES"]

# Drop the specified columns
columns_to_drop = ["order", "taxonKey", "acceptedTaxonKey", "orderKey", "family", "familyKey"]
named_GBIF_df = named_GBIF_df.drop(columns=columns_to_drop)

#print(named_GBIF_df)
named_GBIF_df.to_csv("3b_GBIF_condensed_named_file.csv", index=False)



In [155]:
condensed_GBIF_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3b_GBIF_condensed_named_file.csv")

# Filter rows where commonName is null or empty
missing_common_name_rows = condensed_GBIF_df[condensed_GBIF_df["commonName"].isnull() | (condensed_GBIF_df["commonName"] == "")]

# Print results
print(f"Number of rows in total: {len(condensed_GBIF_df)}")
print(f"Number of rows without common names: {len(missing_common_name_rows)}")
print("SplitScientificNames and their number of occurrences:")
print(missing_common_name_rows[["splitScientificName","taxonRank","numberOfOccurrences"]])




Number of rows in total: 644
Number of rows without common names: 7
SplitScientificNames and their number of occurrences:
             splitScientificName taxonRank  numberOfOccurrences
18              Dryobates dignus   SPECIES                    6
86   Xiphorhynchus aequatorialis   SPECIES                    2
241       Phylloscartes flaveola   SPECIES                    2
296            Tangara aurulenta   SPECIES                    4
299             Tangara lunigera   SPECIES                    1
307          Buthraupis edwardsi   SPECIES                    2
339       Sporophila ophthalmica   SPECIES                    2


In [156]:
#Get Clements list ready to merge 
# File path to the CSV file
csv_file = "C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\Clements-v2024-October-2024-rev.csv"  # Replace with your actual file path

# Import only the specified columns and filter rows
Clements_df = pd.read_csv(
    csv_file,
    usecols=["sort v2024", "species_code", "English name", "scientific name", "category"]
)

# Filter rows where "category" is set to "species"
Clements_filtered_df = Clements_df[Clements_df["category"] == "species"]

# Drop the "category" column if not needed
Clements_filtered_df = Clements_filtered_df.drop(columns=["category"])



# Save the filtered DataFrame to a new CSV file (optional)
Clements_filtered_df.to_csv("4_Clements_filtered_species.csv", index=False)
print("Filtered data saved to '4_Clements_filtered_species.csv'.")

Filtered data saved to '4_Clements_filtered_species.csv'.


In [157]:
# Merge the GBIF and Clements
# Use 'inner', 'left', 'right', or 'outer' for different types of joins

# Import only the specified columns and filter rows
condensed_GBIF_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\3b_GBIF_condensed_named_file.csv")
Clements_filtered_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\4_Clements_filtered_species.csv")


# First pass: Match by scientific name
merged_df = pd.merge(
    Clements_filtered_df,
    condensed_GBIF_df,
    left_on="scientific name",
    right_on="splitScientificName",
    how="right"
)

# Convert the lookup DataFrame's 'English name' column to lowercase for case-insensitive comparison
Clements_filtered_df["English name_lower"] = Clements_filtered_df["English name"].str.lower()
merged_df["commonName_lower"] = merged_df["commonName"].str.lower()


# Iterate over the rows of df and perform the lookup
for index, row in merged_df.iterrows():
    
    # Look for a match in the lookup DataFrame
    match = Clements_filtered_df[Clements_filtered_df["English name_lower"] == row["commonName_lower"]]
    
    # If a match is found, copy the values
    if not match.empty:
        merged_df.at[index, "English name"] = match.iloc[0]["English name"]
        merged_df.at[index, "sort v2024"] = match.iloc[0]["sort v2024"]
        merged_df.at[index, "species_code"] = match.iloc[0]["species_code"]
        merged_df.at[index, "scientific name"] = match.iloc[0]["scientific name"]


# Sort the DataFrame by multiple columns
merged_df = merged_df.sort_values(by=['sort v2024'],ascending=[True])

# Drop the original column if desired
merged_df = merged_df.drop(columns=["commonName_lower"])

# Replace empty or NaN values in 'commonName' with the value from 'English Name'
merged_df["commonName"] = merged_df["commonName"].replace(["", None], pd.NA)  # Standardize empty entries to NaN
merged_df["commonName"] = merged_df["commonName"].fillna(merged_df["English name"])

# Save the combined DataFrame to a new CSV file
merged_df.to_csv("5_Species_merged.csv", index=False)
#merged_df.to_csv("Species_merged.csv", index=False)


print(f"Combined CSV file has been saved as 5_Species_merged.csv.")

Combined CSV file has been saved as 5_Species_merged.csv.


In [4]:
merged_df = pd.read_csv("C:\\Users\\Walter\\Documents\\GitHub\\FeatheredMaps\\5_Species_merged.csv")

# Filter rows where commonName is null or empty
missing_common_name_rows = merged_df[merged_df["commonName"].isnull() | (merged_df["commonName"] == "")]

# Print results
print(f"Numbe of rows: {len(merged_df)}")
print(f"Number of rows without common names: {len(missing_common_name_rows)}")
print("SplitScientificNames and their number of occurrences:")
print(missing_common_name_rows[["splitScientificName","taxonRank","numberOfOccurrences"]])

Numbe of rows: 644
Number of rows without common names: 6
SplitScientificNames and their number of occurrences:
             splitScientificName taxonRank  numberOfOccurrences
630  Xiphorhynchus aequatorialis   SPECIES                    2
632       Phylloscartes flaveola   SPECIES                    2
633            Tangara aurulenta   SPECIES                    4
634             Tangara lunigera   SPECIES                    1
635          Buthraupis edwardsi   SPECIES                    2
638       Sporophila ophthalmica   SPECIES                    2


In [45]:
# Step 1: Calculate the total number of occurrences
total_occurrences = sorted_df['numberOfOccurrences'].sum()

ratio_df = sorted_df.copy()

# Step 2: Calculate the relative abundance (absoluteRatio)
ratio_df['absoluteRatio'] = ratio_df['numberOfOccurrences'] / total_occurrences

# Display the modified DataFrame
#print("DataFrame with 'absoluteRatio' column:")
#print(ratio_df.head())  # Display the first few rows of the modified DataFrame

# Save the modified DataFrame to a new CSV file (optional)
ratio_df.to_csv('rarity.csv', index=False, sep='\t')
print("Modified DataFrame saved to 'rarity.csv'.")

Modified DataFrame saved to 'rarity.csv'.


In [30]:

df=ratio_df

# Ensure the 'absoluteRatio' column exists
if 'absoluteRatio' not in df.columns:
    # Calculate the total number of occurrences
    total_occurrences = df['numberOfOccurrences'].sum()
    # Calculate the relative abundance (absoluteRatio)
    df['absoluteRatio'] = df['numberOfOccurrences'] / total_occurrences

# Define percentiles for IUCN-inspired categories
percentiles = df['absoluteRatio'].quantile([0.95, 0.80, 0.50, 0.20])

# Define a function to categorize rarity based on percentiles
def categorize_rarity(ratio):
    if ratio >= percentiles[0.95]:
        return 'Abundant'
    elif percentiles[0.80] <= ratio < percentiles[0.95]:
        return 'Common'
    elif percentiles[0.50] <= ratio < percentiles[0.80]:
        return 'Uncommon'
    elif percentiles[0.20] <= ratio < percentiles[0.50]:
        return 'Rare'
    else:
        return 'Very Rare'

# Apply the categorization function to create a new column
df['rarity'] = df['absoluteRatio'].apply(categorize_rarity)

# Display the modified DataFrame
print("DataFrame with 'rarity' column:")
print(df.head())  # Display the first few rows of the modified DataFrame

# Save the modified DataFrame to a new CSV file (optional)
df.to_csv('modified_file_with_rarity.csv', index=False, sep='\t')
print("Modified DataFrame saved to 'modified_file_with_rarity.csv'.")

DataFrame with 'rarity' column:
     taxonKey                             scientificName  acceptedTaxonKey  \
150   2482183      Chamaepetes goudotii (R.Lesson, 1828)           2482183   
636   6177862  Chamaepetes goudotii fagani C.Chubb, 1917           6177862   
145   2482198       Penelope montagnii (Bonaparte, 1856)           2482198   
491   2482216               Penelope ortoni Salvin, 1874           2482216   
48    2482225         Penelope purpurascens Wagler, 1830           2482225   

                        acceptedScientificName  numberOfOccurrences  \
150      Chamaepetes goudotii (R.Lesson, 1828)                  740   
636  Chamaepetes goudotii fagani C.Chubb, 1917                    2   
145       Penelope montagnii (Bonaparte, 1856)                   25   
491               Penelope ortoni Salvin, 1874                    8   
48          Penelope purpurascens Wagler, 1830                 3098   

      taxonRank taxonomicStatus        order  orderKey    family  family

In [40]:

# Step 1: Calculate the total numberOfOccurrences for each speciesKey
df['totalOccurrences'] = df.groupby('genusKey')['numberOfOccurrences'].transform('sum')

# Step 2: Calculate the relativeRatio
df['relativeRatio'] = df['numberOfOccurrences'] / df['totalOccurrences']

# Define a function to format percentages dynamically
def format_percent(value):
    if value >= 10:
        return f"{value:.1f}%"  # Keep 1 decimal place for values >= 10
    elif value >= 1:
        return f"{value:.2f}%"  # Keep 2 decimal places for values >= 1
    else:
        return f"{value:.3f}%"  # Keep 3 decimal places for values < 1

# Convert ratios to percentages and format them
df['relativePercent'] = (df['relativeRatio'] * 100).apply(format_percent)



df = df.drop(columns=['totalOccurrences'])


# Display the modified DataFrame
print("DataFrame with 'relativeRatio' column:")
print(df.head())  # Display the first few rows of the modified DataFrame

# Save the modified DataFrame to a new CSV file (optional)
df.to_csv('modified_file_with_relativeRatio.csv', index=False, sep='\t')
print("Modified DataFrame saved to 'modified_file_with_relativeRatio.csv'.")



DataFrame with 'relativeRatio' column:
     taxonKey  acceptedTaxonKey                     acceptedScientificName  \
150   2482183           2482183      Chamaepetes goudotii (R.Lesson, 1828)   
636   6177862           6177862  Chamaepetes goudotii fagani C.Chubb, 1917   
145   2482198           2482198       Penelope montagnii (Bonaparte, 1856)   
491   2482216           2482216               Penelope ortoni Salvin, 1874   
48    2482225           2482225         Penelope purpurascens Wagler, 1830   

     numberOfOccurrences   taxonRank taxonomicStatus        order  orderKey  \
150                  740     SPECIES        ACCEPTED  Galliformes     723.0   
636                    2  SUBSPECIES        ACCEPTED  Galliformes     723.0   
145                   25     SPECIES        ACCEPTED  Galliformes     723.0   
491                    8     SPECIES        ACCEPTED  Galliformes     723.0   
48                  3098     SPECIES        ACCEPTED  Galliformes     723.0   

       family  fa

In [35]:
# Filter rows where taxonRank is 'SUBSPECIES'
subspecies_df = df[df['taxonRank'] == 'SUBSPECIES']

# Save the filtered DataFrame to a CSV file
subspecies_df.to_csv('subspecies.csv', index=False)

print("Filtered DataFrame has been saved as 'subspecies.csv'.")

Filtered DataFrame has been saved as 'subspecies.csv'.
